In [0]:

# Check if the secret exists and retrieve it
storage_key = dbutils.secrets.get(
    scope = 'databricksScope',
    key = 'storageAccountkey'
)

# Set the Spark config with the correct key
spark.conf.set(
    "fs.azure.account.key.medallionsta.dfs.core.windows.net",
    storage_key
)


dbutils.widgets.text('fileName', '20250926')
dbutils.widgets.text('table_schema', 'SalesLT')
dbutils.widgets.text('table_name', 'Customer')

fileName = dbutils.widgets.get('fileName')
tableSchema = dbutils.widgets.get('table_schema')
tableName = dbutils.widgets.get('table_name')

print(f"Parameters received:")
print(f"fileName: {fileName}")
print(f"tableSchema: {tableSchema}")
print(f"tableName: {tableName}")

try:
    spark.sql(f"CREATE DATABASE IF NOT EXISTS `{tableSchema}`")
    print(f"Database {tableSchema} created or already exists")
    
    adls_path = (
        f"abfss://bronze@medallionsta.dfs.core.windows.net/"
        f"{fileName}/{tableSchema}.{tableName}.parquet"
    )
    print(f"Table location: {adls_path}")

    table_schema_path = f"`{tableSchema}`.`{tableName}`"

    # Register the Parquet file as an external table
    spark.sql(
        f"""
        CREATE TABLE IF NOT EXISTS {table_schema_path}
        USING PARQUET
        LOCATION '{adls_path}'
        """
    )
    print(f"Table {tableSchema}.{tableName} registered as external table at {adls_path}")

except Exception as e:
    print(f"Error creating database or table: {str(e)}")
    raise

Parameters received:
fileName: 20250926
tableSchema: SalesLT
tableName: Customer
Database SalesLT created or already exists
Table location: abfss://bronze@medallionsta.dfs.core.windows.net/20250926/SalesLT.Customer.parquet
Table SalesLT.Customer registered as external table at abfss://bronze@medallionsta.dfs.core.windows.net/20250926/SalesLT.Customer.parquet
